A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
%matplotlib inline

In [5]:
dataset=pd.read_csv("/Users/piyush/MLPROJECT/PredictingMovieRentalDurations/rental_info.csv")

In [6]:
dataset

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15856,2005-08-22 10:49:15+00:00,2005-08-29 09:52:15+00:00,6.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,48.8601,7744.0,24.9001
15857,2005-07-31 09:48:49+00:00,2005-08-04 10:53:49+00:00,4.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,24.9001,7744.0,24.9001
15858,2005-08-20 10:35:30+00:00,2005-08-29 13:03:30+00:00,8.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,80.8201,7744.0,24.9001
15859,2005-07-31 13:10:20+00:00,2005-08-08 14:07:20+00:00,7.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,63.8401,7744.0,24.9001


In [7]:
dataset.keys()

Index(['rental_date', 'return_date', 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost', 'special_features', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2'],
      dtype='object')

Create a column named "rental_length_days" using the columns "return_date" and "rental_date", and add it to the pandas DataFrame.
This column should contain information on how many days a DVD has been rented by a customer.

In [8]:
dataset["rental_length"]=pd.to_datetime(dataset["return_date"])-pd.to_datetime(dataset["rental_date"])
dataset["rental_length_days"]=dataset["rental_length"].dt.days

In [9]:
dataset.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length,rental_length_days
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3 days 20:46:00,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 20:05:00,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7 days 05:44:00,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 02:24:00,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4 days 01:05:00,4


Create two columns of dummy variables from "special_features", which takes the value of 1 when:
The value is "Deleted Scenes", storing as a column called "deleted_scenes".
The value is "Behind the Scenes", storing as a column called "behind_the_scenes".

In [10]:
##Add dummy for deleted scenes
dataset["deleted_scenes"]=np.where(dataset["special_features"].str.contains("Deleted Scenes"),1,0)
##Add dummy for behind the scenes
dataset["behind_the_scenes"]=np.where(dataset["special_features"].str.contains("Behind the Scenes"),1,0)

In [11]:
dataset

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length,rental_length_days,deleted_scenes,behind_the_scenes
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3 days 20:46:00,3,0,1
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 20:05:00,2,0,1
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7 days 05:44:00,7,0,1
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2 days 02:24:00,2,0,1
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4 days 01:05:00,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15856,2005-08-22 10:49:15+00:00,2005-08-29 09:52:15+00:00,6.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,48.8601,7744.0,24.9001,6 days 23:03:00,6,1,1
15857,2005-07-31 09:48:49+00:00,2005-08-04 10:53:49+00:00,4.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,24.9001,7744.0,24.9001,4 days 01:05:00,4,1,1
15858,2005-08-20 10:35:30+00:00,2005-08-29 13:03:30+00:00,8.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,80.8201,7744.0,24.9001,9 days 02:28:00,9,1,1
15859,2005-07-31 13:10:20+00:00,2005-08-08 14:07:20+00:00,7.99,2009.0,4.99,88.0,11.99,"{Trailers,""Deleted Scenes"",""Behind the Scenes""}",0,0,0,1,63.8401,7744.0,24.9001,8 days 00:57:00,8,1,1


In [12]:
cols_to_drop=["rental_length","rental_date","return_date","rental_length_days","special_features"]
X=dataset.drop(cols_to_drop,axis=1)
y=dataset["rental_length_days"]

In [13]:
X

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,deleted_scenes,behind_the_scenes
0,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
1,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
2,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
3,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
4,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15856,6.99,2009.0,4.99,88.0,11.99,0,0,0,1,48.8601,7744.0,24.9001,1,1
15857,4.99,2009.0,4.99,88.0,11.99,0,0,0,1,24.9001,7744.0,24.9001,1,1
15858,8.99,2009.0,4.99,88.0,11.99,0,0,0,1,80.8201,7744.0,24.9001,1,1
15859,7.99,2009.0,4.99,88.0,11.99,0,0,0,1,63.8401,7744.0,24.9001,1,1


In [14]:
y

0        3
1        2
2        7
3        2
4        4
        ..
15856    6
15857    4
15858    9
15859    8
15860    6
Name: rental_length_days, Length: 15861, dtype: int64

In [15]:
##Split the data into Train and Test set

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=9)

In [16]:
X_train

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,deleted_scenes,behind_the_scenes
6682,2.99,2010.0,2.99,90.0,25.99,1,0,0,0,8.9401,8100.0,8.9401,0,1
8908,4.99,2008.0,0.99,53.0,25.99,1,0,0,0,24.9001,2809.0,0.9801,1,0
11827,6.99,2007.0,4.99,171.0,25.99,0,0,1,0,48.8601,29241.0,24.9001,0,1
6153,2.99,2010.0,2.99,73.0,29.99,0,0,0,1,8.9401,5329.0,8.9401,1,1
10713,5.99,2004.0,0.99,122.0,14.99,1,0,0,0,35.8801,14884.0,0.9801,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6200,1.99,2007.0,0.99,157.0,27.99,1,0,0,0,3.9601,24649.0,0.9801,1,1
501,6.99,2005.0,4.99,135.0,28.99,0,0,1,0,48.8601,18225.0,24.9001,1,1
6782,5.99,2005.0,4.99,177.0,20.99,1,0,0,0,35.8801,31329.0,24.9001,1,1
4444,2.99,2006.0,2.99,169.0,21.99,0,0,1,0,8.9401,28561.0,8.9401,0,0


In [17]:
X_test

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,deleted_scenes,behind_the_scenes
15067,4.99,2005.0,0.99,184.0,9.99,0,0,1,0,24.9001,33856.0,0.9801,1,1
3808,4.99,2005.0,4.99,179.0,29.99,0,0,0,1,24.9001,32041.0,24.9001,0,1
1015,4.99,2007.0,4.99,73.0,17.99,0,1,0,0,24.9001,5329.0,24.9001,1,1
12617,4.99,2009.0,0.99,172.0,14.99,0,0,0,1,24.9001,29584.0,0.9801,0,1
1711,4.99,2007.0,4.99,91.0,16.99,0,0,1,0,24.9001,8281.0,24.9001,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2828,2.99,2004.0,0.99,129.0,15.99,0,1,0,0,8.9401,16641.0,0.9801,0,1
8917,9.99,2010.0,4.99,134.0,17.99,0,0,1,0,99.8001,17956.0,24.9001,1,1
13592,0.99,2004.0,0.99,134.0,20.99,0,0,0,1,0.9801,17956.0,0.9801,1,1
7739,2.99,2006.0,2.99,141.0,21.99,0,0,1,0,8.9401,19881.0,8.9401,0,0


In [18]:
y_train

6682     4
8908     8
11827    5
6153     0
10713    9
        ..
6200     7
501      6
6782     6
4444     3
8574     8
Name: rental_length_days, Length: 12688, dtype: int64

In [19]:
y_test

15067    8
3808     1
1015     6
12617    9
1711     5
        ..
2828     9
8917     9
13592    5
7739     3
1768     6
Name: rental_length_days, Length: 3173, dtype: int64

In [20]:
##Create the Lasso model

lasso=Lasso(alpha=0.1,random_state=9)

In [21]:
##Train the lasso model and access the coefficients
lasso.fit(X_train,y_train)
lasso_coef=lasso.coef_

In [22]:
## Perform feature selection by choosing columns with positive coefficients
X_lasso_train,X_lasso_test=X_train.iloc[:,lasso_coef>0],X_test.iloc[:,lasso_coef>0]

In [23]:
# Run OLS models on lasso chosen regression
ols=LinearRegression()
ols.fit(X_lasso_train,y_train)
y_test_pred=ols.predict(X_lasso_test)
mse_lin_reg_lasso=mean_squared_error(y_test,y_test_pred)

In [24]:
print(mse_lin_reg_lasso)

4.812297241276236


In [25]:
##Random Forests Hyperparameter space
params_dist={"n_estimators":np.arange(1,101,1),
            "max_depth":np.arange(1,11,1)}

In [26]:
#Create a Random Forest Regressor
rf=RandomForestRegressor()

In [27]:
##Use Random Search to find the best hyperparameters
random_search=RandomizedSearchCV(rf,param_distributions=params_dist,cv=5,random_state=9)

In [28]:
#Fit the random search object to the training data
random_search.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'n_estimators': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])},
                   random_state=9)

In [29]:
#Create a variable for best hyperparameters

hyper_params = random_search.best_params_

In [30]:
hyper_params

{'n_estimators': 51, 'max_depth': 10}

In [31]:
##Run the Random Forest on the chosen hyper parameters
rf=RandomForestRegressor(n_estimators=hyper_params["n_estimators"],
                        max_depth=hyper_params["max_depth"],
                        random_state=9)

In [32]:
rf.fit(X_train,y_train)
rf_test_pred=rf.predict(X_test)
mse_random_forest= mean_squared_error(y_test,rf_test_pred)

In [33]:
mse_random_forest

2.225667528098759

In [34]:
#Random Forest gives lowest MSE :
best_model =rf
best_mse=mse_random_forest

In [35]:
best_model

RandomForestRegressor(max_depth=10, n_estimators=51, random_state=9)

In [36]:
best_mse

2.225667528098759

## Pickling the Model for deployment

In [37]:
import pickle

In [38]:
pickle.dump(rf,open('rfmodel.pkl','wb'))

In [39]:
pickled_model=pickle.load(open('rfmodel.pkl','rb'))

In [40]:
## Prediction
pickled_model.predict(X_test)

array([7.37874925, 2.72296869, 2.15152358, ..., 2.26404143, 2.68315905,
       6.77543634])